In [1]:
import json
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import unicodedata
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker

import seaborn as sns
import matplotlib.pyplot as plt

from google_play_scraper import Sort, reviews, app

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/raycalm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/raycalm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/raycalm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
apps_ids = ['br.com.brainweb.ifood', 'com.cerveceriamodelo.modelonow',

'com.mcdo.mcdonalds', 'habibs.alphacode.com.br',

'com.ubercab.eats', 'com.grability.rappi',

'burgerking.com.br.appandroid', 

'com.vanuatu.aiqfome',

'br.com.sbf.centauro',

'br.com.riachuelo.app',

'com.mercadolibre',

'com.cerveceriamodelo.modelonow',

'com.amazon.mShop.android.shopping',

'br.gov.meugovbr',

'com.b2w.americanas',

'br.com.lojasrenner',

'br.com.marisa.android',

'com.zzkko',

'com.luizalabs.mlapp']

In [4]:
apps_infos = []

for ap in tqdm(apps_ids):
    print(ap)
    info = app(ap, lang='pt', country='br')
    del info['comments']
    apps_infos.append(info)

  0%|          | 0/19 [00:00<?, ?it/s]

br.com.brainweb.ifood


  5%|▌         | 1/19 [00:00<00:07,  2.26it/s]

com.cerveceriamodelo.modelonow


 11%|█         | 2/19 [00:00<00:07,  2.33it/s]

...

 89%|████████▉ | 17/19 [00:06<00:00,  2.79it/s]

com.zzkko


 95%|█████████▍| 18/19 [00:06<00:00,  2.65it/s]

com.luizalabs.mlapp


100%|██████████| 19/19 [00:07<00:00,  2.70it/s]


In [5]:
apps_infos_df = pd.DataFrame(apps_infos)
apps_infos_df.head()

,title,description,descriptionHTML,summary,installs,minInstalls,realInstalls,score,ratings,reviews,...,contentRating,contentRatingDescription,adSupported,containsAds,released,lastUpdatedOn,updated,version,appId,url
0,iFood comida e mercado em casa,"Compre em mercados, restaurantes, farmácias e ...","Compre em mercados, restaurantes, farmácias e ...","Peça delivery de comida, mercado, bebidas, far...",100.000.000+,100000000,118107802,4.777080,13227783,1462545,...,Classificação Livre,None,False,False,25 de abr. de 2012,15 de jul. de 2024,1721079285,Varies with device,br.com.brainweb.ifood,https://play.google.com/store/apps/details?id=...
1,Zé Delivery de Bebidas,Zé Delivery: o maior app de bebidas do país! \...,Zé Delivery: o maior app de bebidas do país! <...,"Delivery de bebidas a preço baixo, com entrega...",10.000.000+,10000000,23892080,4.854219,1239430,258662,...,Classificação 12 anos,"Drogas Legais, Temas sensíveis",False,False,13 de dez. de 2017,7 de mai. de 2024,1715088016,24.17.1,com.cerveceriamodelo.modelonow,https://play.google.com/store/apps/details?id=...
2,McDonald’s: Cupons e Delivery,Bora fazer aquela Méquizice hoje?! 🍔🍦🍟Baixe o ...,Bora fazer aquela Méquizice hoje?! 🍔🍦🍟Baixe o ...,Pega essa dica: Peça seu Méqui no app pelo McD...,50.000.000+,50000000,79041356,4.664683,1593717,92811,...,Classificação Livre,None,True,True,8 de jul. de 2016,27 de jun. de 2024,1719512526,4.1.1,com.mcdo.mcdonalds,https://play.google.com/store/apps/details?id=...
3,Habib's: Descontos e Delivery,"Bora pedir um Habib’s hoje?\r\nAqui no App, vo...","Bora pedir um Habib’s hoje?<br>Aqui no App, vo...",Quer acabar com a sua fome? Vem pro App Habib’s!,5.000.000+,5000000,9522412,4.513964,307293,78264,...,Classificação Livre,None,False,False,30 de jun. de 2019,18 de jun. de 2024,1718713339,3.10.11,habibs.alphacode.com.br,https://play.google.com/store/apps/details?id=...
4,Uber Eats: Delivery de Comida,Está com fome? Receba seus pratos preferidos d...,Está com fome? Receba seus pratos preferidos d...,Cupons de oferta. Delivery de comida de restau...,100.000.000+,100000000,237914195,3.876273,5471999,277859,...,Classificação Livre,None,True,True,6 de jun. de 2016,15 de jul. de 2024,1721059391,Varies with device,com.ubercab.eats,https://play.google.com/store/apps/details?id=...


In [6]:
apps_reviews = []

for ap in tqdm(apps_ids):
    for score in list(range(1, 6)):
        for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
            rvs, _ = reviews(
                ap,
                lang='pt',
                country='br',
                sort=sort_order,
                count= 200 if score == 3 else 100,
                filter_score_with=score
            )
            for r in rvs:
                r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
                r['appId'] = ap
            apps_reviews.extend(rvs)

 11%|█         | 2/19 [00:12<01:47,  6.30s/it]


KeyboardInterrupt: 

In [ ]:
len(apps_reviews)

22800

In [ ]:
apps_reviews_df = pd.DataFrame(apps_reviews)
apps_reviews_df = apps_reviews_df.drop(['reviewId', 'reviewCreatedVersion', 'appVersion', 'repliedAt', 'userImage', 'at'], axis=1)

In [ ]:
apps_reviews_df.head()

,userName,content,score,thumbsUpCount,replyContent,sortOrder,appId
0,Guilherme Gomes,Um lixo de aplicativo. Cada vez mais a experiê...,1,27,None,most_relevant,br.com.brainweb.ifood
1,Ana Luiza,Quanto mais o tempo passa mais inutilizável fi...,1,18,None,most_relevant,br.com.brainweb.ifood
2,Guidi Yu,O atendimento pelo chat no geral era bom. Hoje...,1,23,None,most_relevant,br.com.brainweb.ifood
3,Luis Martins,Estou bem decepcionado com o app. É cada vez m...,1,0,None,most_relevant,br.com.brainweb.ifood
4,Igor Fernandez,"Com certeza um dos piores apps que tenho, volu...",1,66,None,most_relevant,br.com.brainweb.ifood


In [ ]:
apps_reviews_df.to_csv('reviews.csv', index=None, header=True)

In [ ]:
reviews = apps_reviews_df['content'].to_numpy()

### Pré-processando os conteúdos dos arquivos no dicionário

In [ ]:
WORD_MIN_LENGTH = 2
STOP_WORDS_PT = [stop_word.lower() for stop_word in nltk.corpus.stopwords.words('portuguese')]
spell = SpellChecker(language='pt')

def strip_accents(text):
    nfkd = unicodedata.normalize('NFKD', text)
    stripped_text = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return re.sub('[^a-zA-Z]', ' ', stripped_text)

def tokenize(text):
    text = strip_accents(text)
    text = re.sub(re.compile('\n'),' ',text)
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in STOP_WORDS_PT and len(word) >= WORD_MIN_LENGTH]
    return words

def lemmatize(text):
    lemmatized_text = []
    for (word) in text:
        lemmatizing = str(WordNetLemmatizer().lemmatize(word))
        lemmatized_text.append(lemmatizing)
    return lemmatized_text

def preprocess_stl(array):
    preprocessed = []

    stripped_rvs = [strip_accents(rvs) for rvs in array]
    tokenized_rvs = [tokenize(rvs) for rvs in stripped_rvs]
    lemmatized_rvs = [lemmatize(rvs) for rvs in tokenized_rvs]

    preprocessed = lemmatized_rvs

    return preprocessed

def get_unique_words(matrix):
    flattened_array = [item for sublist in matrix for item in sublist]
    unique_words = list(set(flattened_array))
    corrected_unique_words = [spell.correction(word) for word in unique_words]

    return corrected_unique_words

In [ ]:
preprocessed_rvs = preprocess_stl(reviews)
unique_words_rvs = get_unique_words(preprocessed_rvs)

In [ ]:
unique_words_rvs

['autorizam',
 'limimite',
 'fones',
 'alocando',
 'superfluo',
 'baratear',
 'frio',
 'velha',
 'mk',
 'acoes',
 'fluindo',
 'imediatamente',
 'charged',
 'perturbado',
 'chagar',
 'justa',
 'entorno',
 'ajudo',
 'apontaria',
 'happi',
 'esquesie',
 'comoda',
 'note',
 'dmais',
 'acesso',
 'prestativos',
 'constrangedora',
 'optem',
 'toutch',
 'qdo',
 'cumprindo',
 'estendida',
 'scumbags',
 'tooltip',
 'representou',
 'questionam',
 'suuuper',
 'revenda',
 'latrao',
 'habilitada',
 'pontuo',
 'inteira',
 'refris',
 'obrigatoriedade',
 'errando',
 'suspenderam',
 'voucher',
 'vale',
 'tema',
 'reportar',
 'dissabores',
 'negou',
 'invisiveis',
 'demo',
 'mentiram',
 'baldao',
 'certin',
 'informaram',
 'onerava',
 'descongelar',
 'any',
 'lixe',
 'salgadinho',
 'anjun',
 'fraquinho',
 'costa',
 'pergunto',
 'espanto',
 'publico',
 'monnnnte',
 'percebendo',
 'prezamos',
 'atibaia',
 'take',
 'personalizado',
 'mostraram',
 'plenamente',
 'obrigatoriamente',
 'digitando',
 'bam',
 'no